In [42]:
import pandas as pd
import gc
from sklearn import preprocessing
import time
import math
import pickle
import numpy as np

In [43]:
total =  pickle.load(open('../../data/raw/total_high_level.pkl','rb+'))
total = total.reset_index(drop=True)

In [171]:
target_features = total.loc[(total["day"]==7),
                           ["item_brand_id","item_id","user_id","shop_id","is_trade","user_id_shop_id","user_id_item_id","context_timestamp","hour","minute"]]

In [172]:
target_features["minutes"] = (target_features["hour"]*60 + target_features["minute"] + 15)

In [173]:
target_features["minutes_30"] = target_features["minutes"] // 30
target_features["minutes_3"] = target_features["minutes"] // 3

In [174]:
tmp = pd.DataFrame(target_features.groupby(["item_id","minutes_3"])["is_trade"].size()).reset_index()
tmp.columns = ["item_id","minutes_3","item_id_minutes_counts"]
target_features = target_features.reset_index().merge(tmp,on=["item_id","minutes_3"], how="left").set_index("index")

In [175]:
target_features.sort_values(by=['item_id', 'minutes_3'], inplace=True)

In [176]:
target_features['positive_item_id_count'] = target_features.groupby(['item_id','minutes_30'])['item_id_minutes_counts'].diff(1)

In [177]:
target_features['negative_item_id_count'] = target_features.groupby(['item_id','minutes_30'])['item_id_minutes_counts'].diff(-1)

In [178]:
target_features = target_features.sort_index()

In [180]:
del target_features["item_brand_id"]
del target_features["item_id"]
del target_features["user_id"]
del target_features["shop_id"]
del target_features["user_id_shop_id"]
del target_features["user_id_item_id"]
del target_features["context_timestamp"]
del target_features["minute"]
del target_features["hour"]
del target_features["minutes"]
del target_features["minutes_30"]
del target_features["minutes_3"]

In [181]:
target_features.to_pickle('../../data/processed/flow_feature.pkl')

In [182]:
target_features.shape

(2806831, 4)

In [183]:
target_features.head()

,is_trade,item_id_minutes_counts,positive_item_id_count,negative_item_id_count
index,,,,
1488847,0.0,1,0.0,0.0
1488848,0.0,34,2.0,0.0
1488849,0.0,8,2.0,0.0
1488850,0.0,1,0.0,NaN
1488851,0.0,2,-2.0,0.0


In [125]:
tmp = pd.DataFrame(target_features.groupby(["item_id","minutes_10"])["is_trade"].mean()).reset_index()
tmp.columns = ["item_id","minutes_10","cvr"]
target_features = target_features.reset_index().merge(tmp,on=["item_id","minutes_10"], how="left").set_index("index")
tmp = target_features.loc[:,["item_id","item_id_10_minutes_counts","cvr","minutes_10","context_timestamp"]]
tmp.sort_values(by=['item_id', 'minutes_10'], inplace=True)

In [126]:
import matplotlib.pyplot as plt
x = tmp.loc[tmp["item_id"]==9200873080649821191]["minutes_10"]
counts = tmp.loc[tmp["item_id"]==9200873080649821191]["item_id_10_minutes_counts"]
cvr = tmp.loc[tmp["item_id"]==9200873080649821191]["cvr"]

plt.figure(figsize=(20, 6))
fig,left_axis=plt.subplots()  
right_axis = left_axis.twinx()  

p1, = left_axis.plot(x, counts, 'b.-')  
p2, = right_axis.plot(x, cvr, 'r.-')  

left_axis.set_xlabel('10 mintues')  
left_axis.set_ylabel('item_id_10_minutes_counts')  
right_axis.set_ylabel('cvr')  


plt.show()  